In [3]:
import pandas as pd
import numpy as np

# BMI가 추가된 데이터 불러오기 (경로 수정)
train_df = pd.read_csv('../preprocessed_data/train_processed_with_bmi.csv')
test_df = pd.read_csv('../preprocessed_data/test_processed_with_bmi.csv')

def create_cholesterol_category(cholesterol):
    """콜레스테롤 수치에 따라 카테고리를 분류하는 함수 (일반적인 기준)"""
    if cholesterol < 200:
        return 'Normal_Cholesterol'
    elif 200 <= cholesterol < 240:
        return 'Borderline_High_Cholesterol'
    else:
        return 'High_Cholesterol'

def create_glucose_category(glucose):
    """혈당 수치에 따라 카테고리를 분류하는 함수 (일반적인 기준)"""
    if glucose < 100:
        return 'Normal_Glucose'
    elif 100 <= glucose < 126:
        return 'Pre-Diabetic'
    else:
        return 'Diabetic'

# 함수를 사용하여 새로운 파생 변수 생성
train_df['cholesterol_category'] = np.vectorize(create_cholesterol_category)(train_df['cholesterol'])
test_df['cholesterol_category'] = np.vectorize(create_cholesterol_category)(test_df['cholesterol'])

train_df['glucose_category'] = np.vectorize(create_glucose_category)(train_df['glucose'])
test_df['glucose_category'] = np.vectorize(create_glucose_category)(test_df['glucose'])


# 원-핫 인코딩
categorical_cols = ['cholesterol_category', 'glucose_category']
combined_df = pd.concat([train_df.drop('stress_score', axis=1), test_df], ignore_index=True)
combined_df = pd.get_dummies(combined_df, columns=categorical_cols, drop_first=True)

# 다시 train과 test 데이터 분리
train_processed_health = combined_df.iloc[:len(train_df)].copy()
test_processed_health = combined_df.iloc[len(train_df):].copy()
train_processed_health['stress_score'] = train_df['stress_score']

# 새로운 파일로 저장
train_processed_health.to_csv('../preprocessed_data/train_processed_with_health_category.csv', index=False)
test_processed_health.to_csv('../preprocessed_data/test_processed_with_health_category.csv', index=False)

print("건강 등급 파생 변수가 추가된 파일이 'preprocessed_data' 폴더에 저장되었습니다.")

건강 등급 파생 변수가 추가된 파일이 'preprocessed_data' 폴더에 저장되었습니다.


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 최종 전처리된 데이터 불러오기
train_df_health = pd.read_csv('../preprocessed_data/train_processed_with_health_category.csv')
test_df_health = pd.read_csv('../preprocessed_data/test_processed_with_health_category.csv')

# 피처(X)와 타겟(y) 분리
X_health = train_df_health.drop(['ID', 'stress_score'], axis=1)
y_health = train_df_health['stress_score']

# 훈련 데이터와 검증 데이터로 나누기
X_train_health, X_val_health, y_train_health, y_val_health = train_test_split(X_health, y_health, test_size=0.2, random_state=42)

# RandomForestRegressor 모델 초기화 및 학습
model_health = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_health.fit(X_train_health, y_train_health.values.ravel())

# 검증 데이터로 예측
y_pred_health = model_health.predict(X_val_health)

# 모델 성능 평가 (RMSE)
rmse_health = np.sqrt(mean_squared_error(y_val_health, y_pred_health))
print(f"건강 등급 파생 변수 적용 모델의 검증 데이터 RMSE: {rmse_health:.4f}")

# 테스트 데이터로 예측
final_predictions_health = model_health.predict(test_df_health.drop('ID', axis=1))

# 제출 파일 생성
submission_df = pd.read_csv('../data/sample_submission.csv')
submission_df['stress_score'] = final_predictions_health
submission_df.to_csv('submission_with_health_category.csv', index=False)

print("건강 등급 파생 변수가 적용된 모델의 제출 파일 'submission_with_health_category.csv'가 성공적으로 생성되었습니다.")

건강 등급 파생 변수 적용 모델의 검증 데이터 RMSE: 0.2347
건강 등급 파생 변수가 적용된 모델의 제출 파일 'submission_with_health_category.csv'가 성공적으로 생성되었습니다.
